# PGXをJupyter notebookで実行して可視化する

## はじめに

Oracleのグラフネットワーク分析用ツールの[PGX（Parallel Graph AnalytiX）](https://www.oracle.com/technetwork/oracle-labs/parallel-graph-analytix/overview/index.html)を
Jupyterから実行し、出力結果をグラフとしてJupyter notebook上に可視化させる。

PGX実行には、Javaのライブラリを呼び出して使う。
そのためのライブラリとして[JPype](http://jpype.sourceforge.net/)を利用する。

可視化には、vis.jsを利用する。
Jupyterからvis.jsを実行させる方法について詳細は[こちら](https://qiita.com/tmioko/items/b8061a4240511db99dd8)。

参考：   
https://gianniceresa.com/2017/07/pgx-client-tool-language/   
https://blogs.oracle.com/bigdataspatialgraph/using-pgql-in-python

### JVM起動、PGXセッション作成

In [ ]:
from jpype import * 
import glob

In [ ]:
# set a classpath
filenames = glob.glob('/home/miotakei/Applications/pgx-19.1.0/lib/*')
pgx_jar_classpath = ':'.join(filenames)

In [ ]:
# start JVM 
startJVM(getDefaultJVMPath(), "-ea", "-Djava.class.path=" + pgx_jar_classpath )

pgxClass = JClass('oracle.pgx.api.Pgx')

In [ ]:
# create a session on a PGX server
session = pgxClass.createSession('http://localhost:7007', 'session')

# close session
#session.close()

### グラフデータ読み込み、分析、可視化用データ作成

In [ ]:
# read Graph
graph = session.readGraphWithProperties("<path of json file>")

In [ ]:
print(graph)

In [ ]:
# check the graph data
checkgraph = graph.queryPgql(" \
  SELECT  n.name, m.name, n.degree\
  MATCH (n)-[e]->(m) \
  ORDER BY e.times \
  LIMIT 10 \
")

it=checkgraph.getResults().iterator()

while (it.hasNext()):
    element = it.next();
    print(element.toString())

In [ ]:
# analysis
analyst = session.createAnalyst()
dc = analyst.degreeCentrality(graph)

In [ ]:
# check vertex name 
print(graph.getVertexProperties())

In [ ]:
# node data
pgxResultSetNode = graph.queryPgql(" \
  SELECT id(n), n.name, n.prob, e.times \
  MATCH (n) \
               ,(x)-[e]->(y)  \
  WHERE ((n) = (x) OR (n) = (y)) \
  AND e.times >= 100 \
")

it_node = pgxResultSetNode.getResults().iterator()
dict_node = [{"id":i.get(0),"label":i.get(1), "value":i.get(2)} for i in it_node]

# delete duplicate
dict_node = [dict(t) for t in set([tuple(d.items()) for d in dict_node])]

In [ ]:
dict_node

In [ ]:
# edge data
pgxResultSetEdge = graph.queryPgql(" \
  SELECT id(x), id(y), e.times/30 \
  MATCH (x)-[e]->(y) \
  WHERE e.times >= 100 \
")

it_edge = pgxResultSetEdge.getResults().iterator()
dict_edge = [{"from":i.get(0),"to":i.get(1), "width":i.get(2)} for i in it_edge]

In [ ]:
dict_edge

### 可視化用データJSON化、Javascriptのクラス化

In [ ]:
from IPython.display import Javascript
import json

# Transform the graph into a JSON graph
data = {"nodes":dict_node, "edges":dict_edge}
jsonGraph = json.dumps(data, indent=4)

# Send to Javascript
Javascript("""window.jsonGraph={};""".format(jsonGraph))

### HTMLタグ作成
このCellの下にグラフが可視化される。

In [ ]:
%%html
<div id="mynetwork"></div>

### vis.js のjavascript作成

In [ ]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){
    
    // create a network
    var container = document.getElementById('mynetwork');
    var options = {
        width: '1000px',
        height: '800px',
        nodes: {
            shape: 'dot',
            font: {
                face: 'Tahoma'
            }
        },
        edges: {
          arrows: {
            to: {
              enabled: true
            }
          }
        }
    };        
         
    // We load the JSON graph we generated from iPython input
    var graph = window.jsonGraph;
    
    // Display Graph
    var network = new vis.Network(container, graph, options);
});

In [ ]:
#shutdownJVM()
shutdownJVM()